<a href="https://colab.research.google.com/github/raulrosapacheco/Analise-Instacart-CestaDeMercado/blob/main/Analise_Instacart_CestaDeMercado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de Cesta de Mercado

Fonte de Dados: <a href ="https://www.kaggle.com/competitions/instacart-market-basket-analysis/data">Instacart Market Basket Analysis<a>

## Carregando Pacotes

In [4]:
# Imports
import math
import sys, os
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Carregando Dados

In [5]:
aisle = pd.read_csv('/content/drive/MyDrive/Datasets/Instacart/aisles.csv')
department = pd.read_csv('/content/drive/MyDrive/Datasets/Instacart/departments.csv')
order = pd.read_csv('/content/drive/MyDrive/Datasets/Instacart/orders.csv')
product = pd.read_csv('/content/drive/MyDrive/Datasets/Instacart/products.csv')
order_product_prior = pd.read_csv('/content/drive/MyDrive/Datasets/Instacart/order_products__prior.csv')
order_product_train = pd.read_csv('/content/drive/MyDrive/Datasets/Instacart/order_products__train.csv')

## Análise Exploratória

In [26]:
count_array = np.array([['Aisles', aisle.shape[0], aisle.shape[1]],
                        ['Departament', department.shape[0], department.shape[1]],
                        ['Product', product.shape[0], product.shape[1]],
                        ['Order', order.shape[0], order.shape[1]],
                        ['Order_Product_Prior', order_product_prior.shape[0], order_product_prior.shape[1]],
                        ['Order_Product_Train', order_product_train.shape[0], order_product_train.shape[1]]])

pd.DataFrame(count_array, columns = ['DataFrame', 'Número de Registros', 'Número de Variáveis'])

,DataFrame,Número de Registros,Número de Variáveis
0,Aisles,134,2
1,Departament,21,2
2,Product,49688,4
3,Order,3421083,7
4,Order_Product_Prior,32434489,4
5,Order_Product_Train,1384617,4


In [10]:
aisle.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [12]:
department.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [14]:
product.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [19]:
order.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [22]:
order_product_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [24]:
order_product_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [27]:
# Total de registros por categoria de eval_set
order['eval_set'].value_counts()

prior    3214874
train     131209
test       75000
Name: eval_set, dtype: int64

### Merge das Tabelas 


In [28]:
# Merge
order_product_prior = order_product_prior.merge(order, on = 'order_id', how = 'left')
order_product_prior = order_product_prior.merge(product, on = 'product_id', how = 'left')
order_product_prior = order_product_prior.merge(department, on = 'department_id', how = 'left')
order_product_prior = order_product_prior.merge(aisle, on = 'aisle_id', how = 'left')

order_product_prior.head(20)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,department,aisle
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,dairy eggs,eggs
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,produce,fresh vegetables
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,pantry,spices seasonings
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,pantry,oils vinegars
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,pantry,baking ingredients
5,2,17794,6,1,202279,prior,3,5,9,8.0,Carrots,83,4,produce,fresh vegetables
6,2,40141,7,1,202279,prior,3,5,9,8.0,Original Unflavored Gelatine Mix,105,13,pantry,doughs gelatins bake mixes
7,2,1819,8,1,202279,prior,3,5,9,8.0,All Natural No Stir Creamy Almond Butter,88,13,pantry,spreads
8,2,43668,9,0,202279,prior,3,5,9,8.0,Classic Blend Cole Slaw,123,4,produce,packaged vegetables fruits
9,3,33754,1,1,205970,prior,16,5,17,12.0,Total 2% with Strawberry Lowfat Greek Strained...,120,16,dairy eggs,yogurt


In [30]:
order_product_train = order_product_train.merge(order, on = 'order_id', how = 'left')
order_product_train = order_product_train.merge(product, on = 'product_id', how = 'left')
order_product_train = order_product_train.merge(department, on = 'department_id', how = 'left')
order_product_train = order_product_train.merge(aisle, on = 'aisle_id', how = 'left')

order_product_train.head(20)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,department,aisle
0,1,49302,1,1,112108,train,4,4,10,9.0,Bulgarian Yogurt,120,16,dairy eggs,yogurt
1,1,11109,2,1,112108,train,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses
2,1,10246,3,0,112108,train,4,4,10,9.0,Organic Celery Hearts,83,4,produce,fresh vegetables
3,1,49683,4,0,112108,train,4,4,10,9.0,Cucumber Kirby,83,4,produce,fresh vegetables
4,1,43633,5,1,112108,train,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15,canned goods,canned meat seafood
5,1,13176,6,0,112108,train,4,4,10,9.0,Bag of Organic Bananas,24,4,produce,fresh fruits
6,1,47209,7,0,112108,train,4,4,10,9.0,Organic Hass Avocado,24,4,produce,fresh fruits
7,1,22035,8,1,112108,train,4,4,10,9.0,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese
8,36,39612,1,0,79431,train,23,6,18,30.0,Grated Pecorino Romano Cheese,2,16,dairy eggs,specialty cheeses
9,36,19660,2,1,79431,train,23,6,18,30.0,Spring Water,115,7,beverages,water seltzer sparkling water


### Análise de Valores Ausentes

In [31]:
order_product_prior.isnull().sum() / len(order_product_prior) * 100

order_id                  0.00000
product_id                0.00000
add_to_cart_order         0.00000
reordered                 0.00000
user_id                   0.00000
eval_set                  0.00000
order_number              0.00000
order_dow                 0.00000
order_hour_of_day         0.00000
days_since_prior_order    6.40697
product_name              0.00000
aisle_id                  0.00000
department_id             0.00000
department                0.00000
aisle                     0.00000
dtype: float64

In [32]:
order_product_train.isna().sum() / len(order_product_train) * 100

order_id                  0.0
product_id                0.0
add_to_cart_order         0.0
reordered                 0.0
user_id                   0.0
eval_set                  0.0
order_number              0.0
order_dow                 0.0
order_hour_of_day         0.0
days_since_prior_order    0.0
product_name              0.0
aisle_id                  0.0
department_id             0.0
department                0.0
aisle                     0.0
dtype: float64

Vamos manter a variável days_since_prior_order por duas razões:

1- O percentual de valores ausentes é apenas de 6.4%.

2- Não usaremos a variável com o algoritmo MBA.